# Scraping data from 2022

In [ ]:
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
import requests
from playwright.async_api import async_playwright
import time
import numpy as np
pd.set_option('display.max_colwidth', None)

In [ ]:
dates = ["01.12.2022", "02.12.2022", "03.12.2022", "04.12.2022", "05.12.2022", "06.12.2022", "07.12.2022", "08.12.2022", "09.12.2022", "10.12.2022", "11.12.2022", "12.12.2022", "13.12.2022", "14.12.2022", "15.12.2022", "16.12.2022", "17.12.2022", "18.12.2022", "19.12.2022", "20.12.2022", "21.12.2022", "22.12.2022", "23.12.2022", "24.12.2022", "25.12.2022", "26.12.2022", "27.12.2022", "28.12.2022", "29.12.2022", "30.12.2022", "31.12.2022", "01.01.2023", "02.01.2023", "03.01.2023", "04.01.2023"]

In [ ]:
df = pd.read_csv("data/links.csv")

total_df = pd.DataFrame()
df.head(20)

In [ ]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

url = df.iloc[0]['link']
await page.goto(url)

time.sleep(1)
await page.locator("#close-button").click()

In [ ]:
for date in dates:
    df = df.replace(to_replace=r'\d\d.\d\d.202\d', value= date, regex=True)

    for index, row in df.iterrows():
        url = row['link']

        country = row['country']
        area = row['area']
    
        await page.goto(url)

        time.sleep(1)
        await page.get_by_text("PT60M").click()

        time.sleep(1)

        tables = pd.read_html(await page.content(), flavor = 'html5lib')
        data = tables[0]
        data.insert(0, 'date', date)
        data.insert(1, 'country', country)
        data.insert(2, 'area', area)
        total_df = pd.concat([total_df, data], axis=0)
    
    print("scraped", date)
    print("scraped", url)
    total_df

In [ ]:
total_df.columns = ['date','country', 'area', 'mtu', 'price(currency/mwh)', 'price(eur/mwh)']
total_df

# Standardize the missing values
total_df.replace({'-': np.nan}, inplace=True)
total_df.replace({'n/e': np.nan}, inplace=True)

# And reorder the columns
data = total_df[['date', 'country', 'area', 'mtu', 'price(currency/mwh)', 'price(eur/mwh)']]
data.head(50)

In [ ]:
data = data.drop(['price(currency/mwh)'], axis=1)
data

In [ ]:
data.to_csv("data/prices-hourly-111222-201222.csv", mode='a', index=True, header=False)

In [ ]:
df = pd.read_csv("data/prices-hourly-111222-201222.csv")
df['date'] = pd.to_datetime(df['date'],format="%d.%m.%Y")
df = df.sort_values(by=['date', 'country','area','mtu'])
df

In [ ]:
data = df.drop(['Unnamed: 0'], axis=1)
#data = data.drop(['price(currency/mwh)'], axis=1) 
data

In [ ]:
data.to_csv("data/prices-hourly-011222-040123.csv", mode='a', index=False, header=True)

In [ ]:
#data_day = round(data.groupby(['date','country'])['price(eur/mwh)'].mean())
data_day = pd.read_csv("data/prices-hourly-011222-040123.csv")
data_day = round(data.groupby(['date', 'country']).agg({'price(eur/mwh)': 'mean'}))

# Reset the index to get the date on each row.
data_day = data_day.reset_index()
data_day.tail(50)

data_day.to_csv("data/prices-daily-011222-040123.csv", index=False, header=True)